# Data Generation Workbench

## Purpose

This notebook provides an interactive workbench for generating synthetic AM-QADF datasets. Use widgets to configure data generation parameters, preview generated data, and populate MongoDB collections - all without writing code!

## Learning Objectives

By the end of this notebook, you will:
- ✅ Configure STL processing and hatching generation parameters
- ✅ Generate laser parameters, ISPM data, and CT scan data interactively
- ✅ Preview generated data before committing to MongoDB
- ✅ Populate MongoDB collections with generated datasets
- ✅ Understand the relationship between parameters and generated data
- ✅ Create reproducible datasets using random seeds

## Estimated Duration

60-90 minutes

---

## Overview

The Data Generation Workbench enables interactive creation of synthetic AM-QADF datasets:

- 📐 **STL Processing**: Load and process 3D STL models
- 🛤️ **Hatching Generation**: Generate layer-by-layer scan paths
- ⚡ **Laser Parameters**: Create realistic laser process parameters
- 🌡️ **ISPM Data**: Generate in-situ process monitoring data
- 🔬 **CT Scans**: Create computed tomography scan data
- 💾 **MongoDB Population**: Store generated data in the warehouse

---

## Workflow

**Quick Start (Simplest Way):**
1. **Run Setup** (Cell 1): Import libraries and connect to MongoDB
2. **Select STL Model**: Choose an STL file from the dropdown
3. **Click Generate**: All data types (hatching, laser, ISPM, CT scan) will be generated with default parameters

**Advanced Usage:**
- **Configure Parameters**: Adjust parameters in each accordion section before generating
- **Select Collections**: Choose which MongoDB collections to populate (defaults to all)
- **Preview** (optional): Click "Preview" to see what will be generated without saving
- **Load STL** (optional): Click "Load STL" to preview the model and extract metadata

**Note**: The "Generate" button is enabled as soon as you select an STL file. It will automatically load the STL and generate all data types with the current widget parameters (defaults if unchanged).

---

Use the interactive widgets below to explore data generation - all parameters are adjustable in real-time!


In [ ]:
# Setup: Import required libraries
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add parent directory and src directory to path for imports
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
src_dir = project_root / 'src'

# Add project root to path (for src.infrastructure imports)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Add src directory to path (for am_qadf imports)
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Core imports
import ipywidgets as widgets
from ipywidgets import (
    VBox, HBox, Accordion, Tab, Dropdown, RadioButtons, 
    Checkbox, Button, Output, Text, IntSlider, FloatSlider,
    Layout, Box, Label, FloatText, IntText, SelectMultiple,
    HTML as WidgetHTML, interactive, interact, fixed, Textarea
)
from IPython.display import display, Markdown, HTML, clear_output
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from typing import Optional, Tuple, Dict, Any, List, Callable
import json
import time

# Load environment variables from development.env
import os
env_file = project_root / 'development.env'
if env_file.exists():
    with open(env_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                value = value.strip('"\'')
                os.environ[key] = value
    print("✅ Environment variables loaded from development.env")

# Try to import infrastructure
INFRASTRUCTURE_AVAILABLE = False
try:
    from src.infrastructure.database import get_connection_manager
    from src.infrastructure.config import MongoDBConfig
    INFRASTRUCTURE_AVAILABLE = True
except Exception as e:
    print(f"⚠️ Infrastructure layer not available: {type(e).__name__}")

# Try to import generation modules
GENERATION_AVAILABLE = False
try:
    from generation.process.stl_processor import STLProcessor
    from generation.process.hatching_generator import HatchingGenerator, HatchingConfig
    from generation.sensors.laser_parameter_generator import LaserParameterGenerator
    from generation.sensors.ispm_generator import ISPMGenerator
    from generation.sensors.ct_scan_generator import CTScanGenerator
    GENERATION_AVAILABLE = True
except ImportError as e:
    print(f"⚠️ Generation modules not available: {e}")
    GENERATION_AVAILABLE = False

# Initialize MongoDB connection
mongo_client = None
if INFRASTRUCTURE_AVAILABLE:
    try:
        manager = get_connection_manager(env_name="development")
        mongo_client = manager.get_mongodb_client()
        if mongo_client and mongo_client.is_connected():
            print("✅ MongoDB connection established")
        else:
            print("⚠️ MongoDB connection failed")
    except Exception as e:
        print(f"⚠️ MongoDB connection error: {type(e).__name__}: {e}")
        print("   Using demo mode with synthetic data")
else:
    print("⚠️ Using demo mode - infrastructure layer unavailable")

print("✅ Setup complete!")


## Interactive Data Generation Interface

Use the widgets below to configure data generation parameters, preview generated data, and populate MongoDB collections. The interface is organized into configuration panels, visualization areas, and status displays.

**Run the cell below to create the complete interface.**


In [ ]:
# ============================================================================
# COMPLETE WIDGET INTERFACE DEFINITION
# ============================================================================
# This cell defines all widgets and assembles the complete interface
# ============================================================================

# Global state
generated_data = {
    'stl_model': None,
    'hatching_data': None,
    'laser_params': None,
    'ispm_data': None,
    'ct_scan': None,
    'model_id': None
}

# ============================================
# Section 1: Top Panel - Generation Mode and Actions
# ============================================

# Generation mode selector
generation_mode = RadioButtons(
    options=[
        ('Single Model', 'single'),
        ('Batch', 'batch'),
        ('Custom Scenario', 'custom'),
        ('Quick Demo', 'demo')
    ],
    value='single',
    description='Mode:',
    layout=Layout(width='200px')
)

# STL model selector
stl_models_dir = project_root / 'generation' / 'models'
available_stl_files = []
if stl_models_dir.exists():
    available_stl_files = sorted([f.name for f in stl_models_dir.glob('*.stl')])

stl_selector = Dropdown(
    options=available_stl_files if available_stl_files else ['No STL files found'],
    value=available_stl_files[0] if available_stl_files else None,
    description='STL Model:',
    layout=Layout(width='300px'),
    disabled=len(available_stl_files) == 0
)

# Action buttons
generate_button = Button(
    description='Generate',
    button_style='success',
    icon='play',
    layout=Layout(width='120px'),
    disabled=True,  # Disabled until ready
    tooltip='Load STL and configure parameters first'
)

preview_button = Button(
    description='Preview',
    button_style='info',
    icon='eye',
    layout=Layout(width='120px'),
    tooltip='Preview generation configuration'
)

clear_preview_button = Button(
    description='Clear Preview',
    button_style='warning',
    icon='trash',
    layout=Layout(width='140px')
)

save_config_button = Button(
    description='Save Config',
    button_style='',
    icon='save',
    layout=Layout(width='140px')
)

# Status label
generate_status_label = WidgetHTML(
    value='<i style="color:gray; font-size:0.9em;">⚠️ Load STL file to enable Generate</i>',
    layout=Layout(margin='0 10px')
)

# Top panel layout
top_panel = VBox([
    HBox([
        generation_mode,
        stl_selector,
        generate_button,
        preview_button,
        clear_preview_button,
        save_config_button
    ], layout=Layout(
        justify_content='flex-start',
        align_items='center',
        padding='5px'
    )),
    HBox([generate_status_label], layout=Layout(padding='5px'))
], layout=Layout(
    padding='10px',
    border='1px solid #ddd',
    margin='5px'
))

# ============================================
# Section 2: Left Panel - Configuration Accordion
# ============================================

# --- STL Processing Configuration ---
stl_load_button = Button(
    description='Load STL',
    button_style='primary',
    icon='upload',
    layout=Layout(width='150px')
)

stl_preview_toggle = Checkbox(
    value=True,
    description='Show 3D Preview',
    layout=Layout(width='150px')
)

stl_extract_metadata = Checkbox(
    value=True,
    description='Extract Metadata',
    layout=Layout(width='150px')
)

stl_calc_bbox = Checkbox(
    value=True,
    description='Calculate BBox',
    layout=Layout(width='150px')
)

stl_calc_volume = Checkbox(
    value=True,
    description='Calculate Volume',
    layout=Layout(width='150px')
)

stl_metadata_display = Textarea(
    value='',
    description='Metadata:',
    layout=Layout(width='100%', height='150px'),
    disabled=True
)

stl_config_panel = VBox([
    WidgetHTML("<b>STL Processing Configuration</b>"),
    HBox([stl_load_button, stl_preview_toggle]),
    HBox([stl_extract_metadata, stl_calc_bbox, stl_calc_volume]),
    stl_metadata_display
], layout=Layout(padding='10px', border='1px solid #ccc', margin='5px'))

# --- Hatching Generation Configuration ---
layer_thickness = FloatSlider(
    value=0.05,
    min=0.01,
    max=0.5,
    step=0.01,
    description='Layer Thickness (mm):',
    layout=Layout(width='100%')
)

hatch_spacing = FloatSlider(
    value=0.1,
    min=0.05,
    max=0.5,
    step=0.01,
    description='Hatch Spacing (mm):',
    layout=Layout(width='100%')
)

hatch_angle = FloatSlider(
    value=45.0,
    min=0.0,
    max=180.0,
    step=5.0,
    description='Hatch Angle (deg):',
    layout=Layout(width='100%')
)

contour_offset = FloatSlider(
    value=0.0,
    min=0.0,
    max=0.5,
    step=0.01,
    description='Contour Offset (mm):',
    layout=Layout(width='100%')
)

max_layers = IntSlider(
    value=50,
    min=1,
    max=1000,
    step=1,
    description='Max Layers:',
    layout=Layout(width='100%')
)

sampling_spacing = FloatSlider(
    value=1.0,
    min=0.1,
    max=10.0,
    step=0.1,
    description='Sampling Spacing (mm):',
    layout=Layout(width='100%')
)

laser_beam_width = FloatSlider(
    value=0.1,
    min=0.01,
    max=0.5,
    step=0.01,
    description='Laser Beam Width (mm):',
    layout=Layout(width='100%')
)

hatch_pattern = RadioButtons(
    options=[('Standard', 'standard'), ('Island', 'island'), ('Contour', 'contour')],
    value='standard',
    description='Pattern:',
    layout=Layout(width='100%')
)

preview_hatching_button = Button(
    description='Preview Hatching',
    button_style='info',
    icon='eye',
    layout=Layout(width='180px')
)

hatching_stats_display = WidgetHTML(value='<i>No hatching generated yet</i>')

hatching_config_panel = VBox([
    WidgetHTML("<b>Hatching Generation Configuration</b>"),
    layer_thickness,
    hatch_spacing,
    hatch_angle,
    contour_offset,
    max_layers,
    sampling_spacing,
    laser_beam_width,
    hatch_pattern,
    HBox([preview_hatching_button]),
    hatching_stats_display
], layout=Layout(padding='10px', border='1px solid #ccc', margin='5px'))

# --- Laser Parameters Configuration ---
power_mean = FloatSlider(
    value=250.0,
    min=50.0,
    max=500.0,
    step=5.0,
    description='Mean Power (W):',
    layout=Layout(width='100%')
)

power_std = FloatSlider(
    value=25.0,
    min=0.0,
    max=100.0,
    step=1.0,
    description='Std Power (W):',
    layout=Layout(width='100%')
)

power_min = FloatSlider(
    value=100.0,
    min=50.0,
    max=400.0,
    step=5.0,
    description='Min Power (W):',
    layout=Layout(width='100%')
)

power_max = FloatSlider(
    value=400.0,
    min=100.0,
    max=500.0,
    step=5.0,
    description='Max Power (W):',
    layout=Layout(width='100%')
)

speed_mean = FloatSlider(
    value=500.0,
    min=100.0,
    max=3000.0,
    step=50.0,
    description='Mean Speed (mm/s):',
    layout=Layout(width='100%')
)

speed_std = FloatSlider(
    value=50.0,
    min=0.0,
    max=200.0,
    step=5.0,
    description='Std Speed (mm/s):',
    layout=Layout(width='100%')
)

speed_min = FloatSlider(
    value=100.0,
    min=50.0,
    max=2000.0,
    step=50.0,
    description='Min Speed (mm/s):',
    layout=Layout(width='100%')
)

speed_max = FloatSlider(
    value=2000.0,
    min=500.0,
    max=3000.0,
    step=50.0,
    description='Max Speed (mm/s):',
    layout=Layout(width='100%')
)

hatch_spacing_mean = FloatSlider(
    value=0.1,
    min=0.05,
    max=0.5,
    step=0.01,
    description='Mean Hatch Spacing (mm):',
    layout=Layout(width='100%')
)

hatch_spacing_std = FloatSlider(
    value=0.01,
    min=0.0,
    max=0.05,
    step=0.001,
    description='Std Hatch Spacing (mm):',
    layout=Layout(width='100%')
)

contour_power_mult = FloatSlider(
    value=1.2,
    min=0.5,
    max=2.0,
    step=0.1,
    description='Contour Power Mult:',
    layout=Layout(width='100%')
)

contour_speed_mult = FloatSlider(
    value=0.8,
    min=0.5,
    max=2.0,
    step=0.1,
    description='Contour Speed Mult:',
    layout=Layout(width='100%')
)

support_power_mult = FloatSlider(
    value=0.7,
    min=0.5,
    max=2.0,
    step=0.1,
    description='Support Power Mult:',
    layout=Layout(width='100%')
)

support_speed_mult = FloatSlider(
    value=1.2,
    min=0.5,
    max=2.0,
    step=0.1,
    description='Support Speed Mult:',
    layout=Layout(width='100%')
)

laser_random_seed = IntText(
    value=None,
    description='Random Seed:',
    layout=Layout(width='200px')
)

preview_laser_button = Button(
    description='Preview Laser Params',
    button_style='info',
    icon='zap',
    layout=Layout(width='180px')
)

laser_config_panel = VBox([
    WidgetHTML("<b>Laser Parameters Configuration</b>"),
    WidgetHTML("<i>Power Configuration</i>"),
    power_mean, power_std, power_min, power_max,
    WidgetHTML("<i>Speed Configuration</i>"),
    speed_mean, speed_std, speed_min, speed_max,
    WidgetHTML("<i>Hatch Spacing</i>"),
    hatch_spacing_mean, hatch_spacing_std,
    WidgetHTML("<i>Region Multipliers</i>"),
    contour_power_mult, contour_speed_mult,
    support_power_mult, support_speed_mult,
    HBox([laser_random_seed, preview_laser_button])
], layout=Layout(padding='10px', border='1px solid #ccc', margin='5px'))

# --- ISPM Data Configuration ---
base_temperature = FloatSlider(
    value=1700.0,
    min=1000.0,
    max=2500.0,
    step=10.0,
    description='Base Temperature (K):',
    layout=Layout(width='100%')
)

temperature_variation = FloatSlider(
    value=100.0,
    min=0.0,
    max=500.0,
    step=5.0,
    description='Temperature Variation (K):',
    layout=Layout(width='100%')
)

melt_pool_width_mean = FloatSlider(
    value=0.5,
    min=0.1,
    max=2.0,
    step=0.05,
    description='Melt Pool Width Mean (mm):',
    layout=Layout(width='100%')
)

melt_pool_width_std = FloatSlider(
    value=0.1,
    min=0.0,
    max=0.5,
    step=0.01,
    description='Width Std (mm):',
    layout=Layout(width='100%')
)

melt_pool_length_mean = FloatSlider(
    value=1.0,
    min=0.2,
    max=5.0,
    step=0.05,
    description='Length Mean (mm):',
    layout=Layout(width='100%')
)

melt_pool_length_std = FloatSlider(
    value=0.15,
    min=0.0,
    max=0.5,
    step=0.01,
    description='Length Std (mm):',
    layout=Layout(width='100%')
)

melt_pool_depth_mean = FloatSlider(
    value=0.3,
    min=0.1,
    max=1.0,
    step=0.05,
    description='Depth Mean (mm):',
    layout=Layout(width='100%')
)

melt_pool_depth_std = FloatSlider(
    value=0.05,
    min=0.0,
    max=0.2,
    step=0.01,
    description='Depth Std (mm):',
    layout=Layout(width='100%')
)

cooling_rate_mean = FloatSlider(
    value=100.0,
    min=10.0,
    max=500.0,
    step=5.0,
    description='Cooling Rate Mean (K/s):',
    layout=Layout(width='100%')
)

cooling_rate_std = FloatSlider(
    value=20.0,
    min=0.0,
    max=100.0,
    step=1.0,
    description='Cooling Rate Std (K/s):',
    layout=Layout(width='100%')
)

temp_gradient_mean = FloatSlider(
    value=50.0,
    min=10.0,
    max=200.0,
    step=5.0,
    description='Temp Gradient Mean (K/mm):',
    layout=Layout(width='100%')
)

temp_gradient_std = FloatSlider(
    value=10.0,
    min=0.0,
    max=50.0,
    step=1.0,
    description='Temp Gradient Std (K/mm):',
    layout=Layout(width='100%')
)

sampling_rate = FloatSlider(
    value=1000.0,
    min=100.0,
    max=10000.0,
    step=100.0,
    description='Sampling Rate (Hz):',
    layout=Layout(width='100%')
)

points_per_layer = IntSlider(
    value=100,
    min=10,
    max=1000,
    step=10,
    description='Points per Layer:',
    layout=Layout(width='100%')
)

ispm_random_seed = IntText(
    value=None,
    description='Random Seed:',
    layout=Layout(width='200px')
)

preview_thermal_button = Button(
    description='Preview ISPM Data',
    button_style='info',
    icon='thermometer',
    layout=Layout(width='180px')
)

ispm_config_panel = VBox([
    WidgetHTML("<b>ISPM Data Configuration</b>"),
    WidgetHTML("<i>Temperature Configuration</i>"),
    base_temperature, temperature_variation,
    WidgetHTML("<i>Melt Pool Size</i>"),
    melt_pool_width_mean, melt_pool_width_std,
    melt_pool_length_mean, melt_pool_length_std,
    melt_pool_depth_mean, melt_pool_depth_std,
    WidgetHTML("<i>Thermal Parameters</i>"),
    cooling_rate_mean, cooling_rate_std,
    temp_gradient_mean, temp_gradient_std,
    WidgetHTML("<i>Sampling</i>"),
    sampling_rate, points_per_layer,
    HBox([ispm_random_seed, preview_thermal_button])
], layout=Layout(padding='10px', border='1px solid #ccc', margin='5px'))

# --- CT Scan Configuration ---
ct_x_dim = IntSlider(
    value=100,
    min=10,
    max=500,
    step=10,
    description='X Dimension:',
    layout=Layout(width='100%')
)

ct_y_dim = IntSlider(
    value=100,
    min=10,
    max=500,
    step=10,
    description='Y Dimension:',
    layout=Layout(width='100%')
)

ct_z_dim = IntSlider(
    value=100,
    min=10,
    max=500,
    step=10,
    description='Z Dimension:',
    layout=Layout(width='100%')
)

ct_x_spacing = FloatSlider(
    value=0.1,
    min=0.01,
    max=1.0,
    step=0.01,
    description='X Spacing (mm):',
    layout=Layout(width='100%')
)

ct_y_spacing = FloatSlider(
    value=0.1,
    min=0.01,
    max=1.0,
    step=0.01,
    description='Y Spacing (mm):',
    layout=Layout(width='100%')
)

ct_z_spacing = FloatSlider(
    value=0.1,
    min=0.01,
    max=1.0,
    step=0.01,
    description='Z Spacing (mm):',
    layout=Layout(width='100%')
)

base_density = FloatSlider(
    value=8.0,
    min=1.0,
    max=20.0,
    step=0.1,
    description='Base Density (g/cm³):',
    layout=Layout(width='100%')
)

density_variation = FloatSlider(
    value=0.2,
    min=0.0,
    max=2.0,
    step=0.05,
    description='Density Variation:',
    layout=Layout(width='100%')
)

base_porosity = FloatSlider(
    value=0.01,
    min=0.0,
    max=0.1,
    step=0.001,
    description='Base Porosity:',
    layout=Layout(width='100%')
)

porosity_variation = FloatSlider(
    value=0.005,
    min=0.0,
    max=0.05,
    step=0.001,
    description='Porosity Variation:',
    layout=Layout(width='100%')
)

defect_probability = FloatSlider(
    value=0.01,
    min=0.0,
    max=0.1,
    step=0.001,
    description='Defect Probability:',
    layout=Layout(width='100%')
)

defect_size_min = IntSlider(
    value=2,
    min=1,
    max=50,
    step=1,
    description='Defect Size Min (voxels):',
    layout=Layout(width='100%')
)

defect_size_max = IntSlider(
    value=10,
    min=2,
    max=100,
    step=1,
    description='Defect Size Max (voxels):',
    layout=Layout(width='100%')
)

defect_density_reduction = FloatSlider(
    value=0.3,
    min=0.0,
    max=0.8,
    step=0.05,
    description='Defect Density Reduction:',
    layout=Layout(width='100%')
)

noise_level = FloatSlider(
    value=0.05,
    min=0.0,
    max=0.2,
    step=0.01,
    description='Noise Level:',
    layout=Layout(width='100%')
)

ct_random_seed = IntText(
    value=None,
    description='Random Seed:',
    layout=Layout(width='200px')
)

preview_ct_button = Button(
    description='Preview CT Scan',
    button_style='info',
    icon='cube',
    layout=Layout(width='180px')
)

ct_config_panel = VBox([
    WidgetHTML("<b>CT Scan Configuration</b>"),
    WidgetHTML("<i>Grid Dimensions</i>"),
    ct_x_dim, ct_y_dim, ct_z_dim,
    WidgetHTML("<i>Voxel Spacing</i>"),
    ct_x_spacing, ct_y_spacing, ct_z_spacing,
    WidgetHTML("<i>Material Properties</i>"),
    base_density, density_variation,
    base_porosity, porosity_variation,
    WidgetHTML("<i>Defect Parameters</i>"),
    defect_probability, defect_size_min, defect_size_max,
    defect_density_reduction,
    WidgetHTML("<i>Scan Quality</i>"),
    noise_level,
    HBox([ct_random_seed, preview_ct_button])
], layout=Layout(padding='10px', border='1px solid #ccc', margin='5px'))

# --- MongoDB Population Options ---
collections_to_populate = SelectMultiple(
    options=[
        ('STL Models', 'stl_models'),
        ('Hatching Layers', 'hatching_layers'),
        ('Laser Parameters', 'laser_parameters'),
        ('ISPM Data', 'ispm_data'),
        ('CT Scan Data', 'ct_scan_data')
    ],
    value=['stl_models', 'hatching_layers', 'laser_parameters', 'ispm_data', 'ct_scan_data'],
    description='Collections:',
    layout=Layout(width='100%')
)

delete_all_toggle = Checkbox(
    value=False,
    description='Delete All Before Populate',
    layout=Layout(width='200px')
)

delete_by_model_id = Text(
    value='',
    placeholder='Enter model_id',
    description='Delete by Model ID:',
    layout=Layout(width='300px')
)

delete_by_model_button = Button(
    description='Delete',
    button_style='danger',
    icon='trash',
    layout=Layout(width='100px')
)

delete_by_collection = Dropdown(
    options=['stl_models', 'hatching_layers', 'laser_parameters', 'ispm_data', 'ct_scan_data'],
    value='stl_models',
    description='Delete Collection:',
    layout=Layout(width='200px')
)

delete_collection_button = Button(
    description='Delete',
    button_style='danger',
    icon='trash',
    layout=Layout(width='100px')
)

batch_size = IntSlider(
    value=100,
    min=1,
    max=1000,
    step=10,
    description='Batch Size:',
    layout=Layout(width='100%')
)

parallel_workers = IntSlider(
    value=4,
    min=1,
    max=20,
    step=1,
    description='Parallel Workers:',
    layout=Layout(width='100%')
)

compress_data = Checkbox(
    value=False,
    description='Compress Data',
    layout=Layout(width='150px')
)

index_collections = Checkbox(
    value=True,
    description='Index Collections',
    layout=Layout(width='150px')
)

store_metadata = Checkbox(
    value=True,
    description='Store Metadata',
    layout=Layout(width='150px')
)

mongodb_config_panel = VBox([
    WidgetHTML("<b>MongoDB Population Options</b>"),
    collections_to_populate,
    WidgetHTML("<i>Delete Options</i>"),
    delete_all_toggle,
    HBox([delete_by_model_id, delete_by_model_button]),
    HBox([delete_by_collection, delete_collection_button]),
    WidgetHTML("<i>Batch Options</i>"),
    batch_size, parallel_workers,
    WidgetHTML("<i>Storage Options</i>"),
    HBox([compress_data, index_collections, store_metadata])
], layout=Layout(padding='10px', border='1px solid #ccc', margin='5px'))

# Combine configuration panels into accordion
config_accordion = Accordion(children=[
    stl_config_panel,
    hatching_config_panel,
    laser_config_panel,
    ispm_config_panel,
    ct_config_panel,
    mongodb_config_panel
], selected_index=None)
config_accordion.set_title(0, 'STL Processing')
config_accordion.set_title(1, 'Hatching Generation')
config_accordion.set_title(2, 'Laser Parameters')
config_accordion.set_title(3, 'ISPM Data')
config_accordion.set_title(4, 'CT Scan')
config_accordion.set_title(5, 'MongoDB Population')

# ============================================
# Section 3: Center Panel - Visualization Tabs
# ============================================

# Create output widgets for each visualization tab
stl_hatching_output = Output(layout=Layout(height='500px', border='1px solid #ccc'))
laser_params_output = Output(layout=Layout(height='500px', border='1px solid #ccc'))
ispm_output = Output(layout=Layout(height='500px', border='1px solid #ccc'))
ct_scan_output = Output(layout=Layout(height='500px', border='1px solid #ccc'))
preview_output = Output(layout=Layout(height='500px', border='1px solid #ccc'))

# Initialize with placeholder messages
with stl_hatching_output:
    display(HTML("<p><i>Load an STL file and generate hatching to see visualization</i></p>"))
with laser_params_output:
    display(HTML("<p><i>Generate laser parameters to see distribution plots</i></p>"))
with ispm_output:
    display(HTML("<p><i>Generate ISPM data to see thermal profiles</i></p>"))
with ct_scan_output:
    display(HTML("<p><i>Generate CT scan data to see volume visualization</i></p>"))
with preview_output:
    display(HTML("<p><i>Click 'Preview' to see combined data preview</i></p>"))

# Create tab widget
visualization_tabs = Tab(children=[
    stl_hatching_output,
    laser_params_output,
    ispm_output,
    ct_scan_output,
    preview_output
])
visualization_tabs.set_title(0, 'STL & Hatching')
visualization_tabs.set_title(1, 'Laser Parameters')
visualization_tabs.set_title(2, 'ISPM Data')
visualization_tabs.set_title(3, 'CT Scan')
visualization_tabs.set_title(4, 'Generation Preview')

# ============================================
# Section 4: Right Panel - Status & Results
# ============================================

# Generation status
current_operation = WidgetHTML(value='<b>Status:</b> Ready')
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    layout=Layout(width='100%')
)

# Step progress indicators
step_status_html = WidgetHTML(value='''
<table style="width:100%">
<tr><td>STL Processing:</td><td>⏳</td></tr>
<tr><td>Hatching:</td><td>⏳</td></tr>
<tr><td>Laser Parameters:</td><td>⏳</td></tr>
<tr><td>ISPM Data:</td><td>⏳</td></tr>
<tr><td>CT Scan:</td><td>⏳</td></tr>
<tr><td>MongoDB Population:</td><td>⏳</td></tr>
</table>
''')

# Generation statistics
gen_stats_html = WidgetHTML(value='''
<b>Generation Statistics</b><br>
<i>No data generated yet</i>
''')

# MongoDB status
mongo_status_html = WidgetHTML(value='''
<b>MongoDB Status</b><br>
<i>Checking connection...</i>
''')

refresh_mongo_button = Button(
    description='Refresh Status',
    button_style='info',
    icon='refresh',
    layout=Layout(width='150px')
)

# Export options
export_config_button = Button(
    description='Export Config',
    button_style='',
    icon='download',
    layout=Layout(width='140px')
)

export_preview_button = Button(
    description='Export Preview',
    button_style='',
    icon='download',
    layout=Layout(width='140px')
)

export_stats_button = Button(
    description='Export Stats',
    button_style='',
    icon='download',
    layout=Layout(width='140px')
)

# Right panel layout
status_panel = VBox([
    WidgetHTML("<b>Generation Status</b>"),
    current_operation,
    progress_bar,
    step_status_html,
    WidgetHTML("<hr>"),
    gen_stats_html,
    WidgetHTML("<hr>"),
    WidgetHTML("<b>MongoDB Status</b>"),
    mongo_status_html,
    refresh_mongo_button,
    WidgetHTML("<hr>"),
    WidgetHTML("<b>Export Options</b>"),
    export_config_button,
    export_preview_button,
    export_stats_button
], layout=Layout(
    padding='10px',
    border='1px solid #ccc',
    margin='5px',
    width='300px'
))

# ============================================
# Section 5: Bottom Panel - Status and Logs
# ============================================

bottom_status = WidgetHTML(value='<b>Status:</b> Ready | <b>Progress:</b> 0% | <b>Time:</b> 0:00')
bottom_progress = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Overall:',
    bar_style='info',
    layout=Layout(width='100%')
)

generation_logs = Output(layout=Layout(
    height='200px',
    border='1px solid #ccc',
    overflow_y='auto'
))

# Initialize logs
with generation_logs:
    display(HTML("<p><i>Generation logs will appear here...</i></p>"))

bottom_panel = VBox([
    bottom_status,
    bottom_progress,
    WidgetHTML("<b>Generation Logs:</b>"),
    generation_logs
], layout=Layout(
    padding='10px',
    border='1px solid #ddd',
    margin='5px'
))

# ============================================
# Complete Interface Layout Assembly
# ============================================

# Main layout: Left (config) + Center (visualization) + Right (status)
main_layout = HBox([
    # Left panel: Configuration accordion (25% width)
    VBox([config_accordion], layout=Layout(width='25%', padding='5px')),
    # Center panel: Visualization tabs (55% width)
    VBox([visualization_tabs], layout=Layout(width='55%', padding='5px')),
    # Right panel: Status and results (20% width)
    VBox([status_panel], layout=Layout(width='20%', padding='5px'))
], layout=Layout(
    width='100%',
    height='600px',
    border='2px solid #333',
    padding='10px'
))

# Complete interface
complete_interface = VBox([
    top_panel,
    main_layout,
    bottom_panel
], layout=Layout(width='100%'))

# Display the complete interface
display(complete_interface)
print("✅ Interface created - ready to use!")


In [ ]:
# ============================================================================
# COMPLETE EVENT HANDLER DEFINITIONS
# ============================================================================
# This cell defines all event handlers that connect widgets to generation logic
# ============================================================================

# Generation start time
generation_start_time = None

# ============================================
# Helper Functions
# ============================================

def log_message(message: str, level: str = 'info'):
    """Log a message to the generation logs."""
    timestamp = datetime.now().strftime('%H:%M:%S')
    icons = {'info': 'ℹ️', 'success': '✅', 'warning': '⚠️', 'error': '❌'}
    icon = icons.get(level, 'ℹ️')
    
    with generation_logs:
        print(f"[{timestamp}] {icon} {message}")

def update_status(operation: str, progress: int = None):
    """Update the status display."""
    current_operation.value = f'<b>Status:</b> {operation}'
    if progress is not None:
        progress_bar.value = progress
        bottom_progress.value = progress
        bottom_status.value = f'<b>Status:</b> {operation} | <b>Progress:</b> {progress}% | <b>Time:</b> {time.strftime("%M:%S", time.gmtime(time.time() - (generation_start_time or time.time())))}'

def update_step_status(step: str, status: str):
    """Update step status indicator."""
    icons = {'pending': '⏳', 'running': '🔄', 'success': '✅', 'error': '❌'}
    icon = icons.get(status, '⏳')
    # Update the step status HTML table
    step_status_html.value = step_status_html.value.replace(
        f'<tr><td>{step}:</td><td>',
        f'<tr><td>{step}:</td><td>{icon}'
    )

def check_generation_ready() -> bool:
    """Check if generation is ready (STL selected)."""
    # Just need an STL file selected - we'll load it automatically if needed
    if not stl_selector.value or stl_selector.value == 'No STL files found':
        return False
    return True

def update_generate_button_state():
    """Update Generate button enabled/disabled state based on readiness."""
    is_ready = check_generation_ready()
    generate_button.disabled = not is_ready
    
    if is_ready:
        generate_button.tooltip = 'Ready to generate all data types with default parameters'
        generate_button.button_style = 'success'
        generate_status_label.value = '<i style="color:green; font-size:0.9em;">✅ Ready to generate - will use default parameters</i>'
    else:
        generate_button.tooltip = 'Select an STL file first'
        generate_button.button_style = ''
        generate_status_label.value = '<i style="color:gray; font-size:0.9em;">⚠️ Select an STL file to enable Generate</i>'

def get_current_config() -> Dict[str, Any]:
    """Collect all current widget values into a configuration dictionary."""
    return {
        'generation_mode': generation_mode.value,
        'stl_file': stl_selector.value,
        'hatching': {
            'layer_thickness': layer_thickness.value,
            'hatch_spacing': hatch_spacing.value,
            'hatch_angle': hatch_angle.value,
            'contour_offset': contour_offset.value,
            'max_layers': max_layers.value,
            'sampling_spacing': sampling_spacing.value,
            'laser_beam_width': laser_beam_width.value,
            'pattern': hatch_pattern.value
        },
        'laser': {
            'power_mean': power_mean.value,
            'power_std': power_std.value,
            'power_min': power_min.value,
            'power_max': power_max.value,
            'speed_mean': speed_mean.value,
            'speed_std': speed_std.value,
            'speed_min': speed_min.value,
            'speed_max': speed_max.value,
            'hatch_spacing_mean': hatch_spacing_mean.value,
            'hatch_spacing_std': hatch_spacing_std.value,
            'contour_power_mult': contour_power_mult.value,
            'contour_speed_mult': contour_speed_mult.value,
            'support_power_mult': support_power_mult.value,
            'support_speed_mult': support_speed_mult.value,
            'random_seed': laser_random_seed.value if laser_random_seed.value else None
        },
        'ispm': {
            'base_temperature': base_temperature.value,
            'temperature_variation': temperature_variation.value,
            'melt_pool_width_mean': melt_pool_width_mean.value,
            'melt_pool_width_std': melt_pool_width_std.value,
            'melt_pool_length_mean': melt_pool_length_mean.value,
            'melt_pool_length_std': melt_pool_length_std.value,
            'melt_pool_depth_mean': melt_pool_depth_mean.value,
            'melt_pool_depth_std': melt_pool_depth_std.value,
            'cooling_rate_mean': cooling_rate_mean.value,
            'cooling_rate_std': cooling_rate_std.value,
            'temp_gradient_mean': temp_gradient_mean.value,
            'temp_gradient_std': temp_gradient_std.value,
            'sampling_rate': sampling_rate.value,
            'points_per_layer': points_per_layer.value,
            'random_seed': ispm_random_seed.value if ispm_random_seed.value else None
        },
        'ct_scan': {
            'grid_dimensions': (ct_x_dim.value, ct_y_dim.value, ct_z_dim.value),
            'voxel_spacing': (ct_x_spacing.value, ct_y_spacing.value, ct_z_spacing.value),
            'base_density': base_density.value,
            'density_variation': density_variation.value,
            'base_porosity': base_porosity.value,
            'porosity_variation': porosity_variation.value,
            'defect_probability': defect_probability.value,
            'defect_size_range': (defect_size_min.value, defect_size_max.value),
            'defect_density_reduction': defect_density_reduction.value,
            'noise_level': noise_level.value,
            'random_seed': ct_random_seed.value if ct_random_seed.value else None
        },
        'mongodb': {
            'collections': list(collections_to_populate.value),
            'delete_all': delete_all_toggle.value,
            'batch_size': batch_size.value,
            'parallel_workers': parallel_workers.value,
            'compress': compress_data.value,
            'index': index_collections.value,
            'store_metadata': store_metadata.value
        }
    }

# ============================================
# Event Handlers
# ============================================

def on_stl_load_clicked(b):
    """Handle STL load button click."""
    if not GENERATION_AVAILABLE:
        log_message("Generation modules not available", 'error')
        return
    
    stl_filename = stl_selector.value
    if not stl_filename or stl_filename == 'No STL files found':
        log_message("No STL file selected", 'warning')
        return
    
    try:
        log_message(f"Loading STL file: {stl_filename}", 'info')
        update_status("Loading STL...", 10)
        
        # Initialize STL processor
        stl_processor = STLProcessor()
        stl_file_path = stl_processor.get_stl_file(stl_filename)
        
        if stl_file_path and stl_file_path.exists():
            # Load STL using pyslm if available
            try:
                import pyslm
                stl_part = pyslm.Part(stl_file_path)
                
                # Extract metadata
                bbox = stl_part.boundingBox
                metadata = {
                    'filename': stl_filename,
                    'path': str(stl_file_path),
                    'bounding_box': {
                        'x_min': bbox[0], 'x_max': bbox[3],
                        'y_min': bbox[1], 'y_max': bbox[4],
                        'z_min': bbox[2], 'z_max': bbox[5]
                    },
                    'volume': stl_part.volume if hasattr(stl_part, 'volume') else None,
                    'loaded_at': datetime.now().isoformat()
                }
                
                generated_data['stl_model'] = stl_part
                generated_data['model_id'] = stl_filename.replace('.stl', '')
                
                # Update metadata display
                metadata_text = json.dumps(metadata, indent=2)
                stl_metadata_display.value = metadata_text
                
                log_message(f"STL loaded successfully: {stl_filename}", 'success')
                update_status("STL loaded", 20)
                update_generate_button_state()
                
                # Show 3D preview if enabled
                if stl_preview_toggle.value:
                    with stl_hatching_output:
                        clear_output(wait=True)
                        display(HTML(f"<p><b>STL Model:</b> {stl_filename}</p>"))
                        display(HTML(f"<p><b>Bounding Box:</b> X: {bbox[0]:.2f} to {bbox[3]:.2f}, "
                                    f"Y: {bbox[1]:.2f} to {bbox[4]:.2f}, "
                                    f"Z: {bbox[2]:.2f} to {bbox[5]:.2f} mm</p>"))
                        if metadata['volume']:
                            display(HTML(f"<p><b>Volume:</b> {metadata['volume']:.2f} mm³</p>"))
                        display(HTML("<p><i>3D visualization would appear here (requires PyVista)</i></p>"))
                
            except ImportError:
                log_message("pyslm not available - limited STL processing", 'warning')
                metadata = {'filename': stl_filename, 'path': str(stl_file_path)}
                stl_metadata_display.value = json.dumps(metadata, indent=2)
                generated_data['stl_model'] = {'path': str(stl_file_path)}
                generated_data['model_id'] = stl_filename.replace('.stl', '')
                update_generate_button_state()
        else:
            log_message(f"STL file not found: {stl_filename}", 'error')
            update_status("Error: STL file not found", 0)
            
    except Exception as e:
        log_message(f"Error loading STL: {str(e)}", 'error')
        update_status(f"Error: {str(e)}", 0)

def on_preview_clicked(b):
    """Handle preview button click - generate and preview data without saving."""
    if not GENERATION_AVAILABLE:
        log_message("Generation modules not available", 'error')
        return
    
    log_message("Starting preview generation...", 'info')
    update_status("Generating preview...", 5)
    
    try:
        config = get_current_config()
        
        # Generate preview data (simplified - would call actual generators)
        with preview_output:
            clear_output(wait=True)
            display(HTML("<h3>Generation Preview</h3>"))
            display(HTML(f"<p><b>Mode:</b> {config['generation_mode']}</p>"))
            display(HTML(f"<p><b>STL File:</b> {config['stl_file']}</p>"))
            display(HTML("<p><b>Configuration Summary:</b></p>"))
            display(HTML(f"<ul>"))
            display(HTML(f"<li>Hatching: {config['hatching']['max_layers']} layers, "
                        f"{config['hatching']['layer_thickness']} mm thickness</li>"))
            display(HTML(f"<li>Laser: {config['laser']['power_mean']} W mean power, "
                        f"{config['laser']['speed_mean']} mm/s mean speed</li>"))
            display(HTML(f"<li>ISPM: {config['ispm']['points_per_layer']} points/layer, "
                        f"{config['ispm']['base_temperature']} K base temp</li>"))
            display(HTML(f"<li>CT Scan: {config['ct_scan']['grid_dimensions']} grid, "
                        f"{config['ct_scan']['base_density']} g/cm³ density</li>"))
            display(HTML(f"</ul>"))
            display(HTML("<p><i>Full preview would show generated data visualizations</i></p>"))
        
        log_message("Preview generated successfully", 'success')
        update_status("Preview ready", 100)
        
    except Exception as e:
        log_message(f"Error generating preview: {str(e)}", 'error')
        update_status(f"Error: {str(e)}", 0)

def on_generate_clicked(b):
    """Handle generate button click - full data generation and MongoDB population."""
    if not GENERATION_AVAILABLE:
        log_message("Generation modules not available", 'error')
        return
    
    if not check_generation_ready():
        log_message("Please select an STL file first", 'warning')
        return
    
    global generation_start_time
    generation_start_time = time.time()
    
    stl_filename = stl_selector.value
    
    # Auto-load STL if not already loaded
    if generated_data.get('stl_model') is None:
        log_message(f"Auto-loading STL file: {stl_filename}", 'info')
        update_status("Loading STL...", 5)
        try:
            stl_processor = STLProcessor()
            stl_file_path = stl_processor.get_stl_file(stl_filename)
            
            if stl_file_path and stl_file_path.exists():
                try:
                    import pyslm
                    stl_part = pyslm.Part(stl_file_path)
                    generated_data['stl_model'] = stl_part
                    generated_data['model_id'] = stl_filename.replace('.stl', '')
                    log_message("STL loaded successfully", 'success')
                except ImportError:
                    generated_data['stl_model'] = {'path': str(stl_file_path)}
                    generated_data['model_id'] = stl_filename.replace('.stl', '')
                    log_message("STL file found (pyslm not available)", 'warning')
            else:
                log_message(f"STL file not found: {stl_filename}", 'error')
                update_status("Error: STL file not found", 0)
                return
        except Exception as e:
            log_message(f"Error loading STL: {str(e)}", 'error')
            update_status(f"Error: {str(e)}", 0)
            return
    
    # Get collections to populate (default to all if none selected)
    collections = list(collections_to_populate.value) if len(collections_to_populate.value) > 0 else [
        'stl_models', 'hatching_layers', 'laser_parameters', 'ispm_data', 'ct_scan_data'
    ]
    
    log_message(f"Starting full data generation for: {stl_filename}", 'info')
    log_message(f"Will generate: {', '.join(collections)}", 'info')
    log_message("Using default parameters from widget settings", 'info')
    update_status("Generating data...", 10)
    
    try:
        model_id = generated_data.get('model_id', stl_filename.replace('.stl', ''))
        config = get_current_config()
        
        # Update progress as we go
        update_step_status("STL Processing", "running")
        update_status("Processing STL...", 20)
        # STL is already loaded above
        
        # Generate hatching if selected
        if 'hatching_layers' in collections:
            update_step_status("STL Processing", "success")
            update_step_status("Hatching", "running")
            update_status("Generating hatching paths...", 30)
            log_message("Generating hatching paths...", 'info')
            
            if GENERATION_AVAILABLE:
                try:
                    hatching_gen = HatchingGenerator()
                    hatching_config = HatchingConfig(
                        layer_thickness=config['hatching']['layer_thickness'],
                        hatch_spacing=config['hatching']['hatch_spacing'],
                        hatch_angle=config['hatching']['hatch_angle'],
                        contour_offset=config['hatching']['contour_offset'],
                        max_layers=config['hatching']['max_layers'],
                        sampling_spacing=config['hatching']['sampling_spacing'],
                        laser_beam_width=config['hatching']['laser_beam_width']
                    )
                    # TODO: Generate hatching data
                    log_message("Hatching generation (placeholder - implement with actual generator)", 'info')
                except Exception as e:
                    log_message(f"Error generating hatching: {str(e)}", 'error')
            
            update_step_status("Hatching", "success")
        
        # Generate laser parameters if selected
        if 'laser_parameters' in collections:
            update_step_status("Laser Parameters", "running")
            update_status("Generating laser parameters...", 50)
            log_message("Generating laser parameters...", 'info')
            
            if GENERATION_AVAILABLE:
                try:
                    laser_gen = LaserParameterGenerator()
                    # TODO: Generate laser parameters with config
                    log_message("Laser parameter generation (placeholder - implement with actual generator)", 'info')
                except Exception as e:
                    log_message(f"Error generating laser parameters: {str(e)}", 'error')
            
            update_step_status("Laser Parameters", "success")
        
        # Generate ISPM data if selected
        if 'ispm_data' in collections:
            update_step_status("ISPM Data", "running")
            update_status("Generating ISPM data...", 70)
            log_message("Generating ISPM data...", 'info')
            
            if GENERATION_AVAILABLE:
                try:
                    ispm_gen = ISPMGenerator()
                    # TODO: Generate ISPM data with config
                    log_message("ISPM data generation (placeholder - implement with actual generator)", 'info')
                except Exception as e:
                    log_message(f"Error generating ISPM data: {str(e)}", 'error')
            
            update_step_status("ISPM Data", "success")
        
        # Generate CT scan if selected
        if 'ct_scan_data' in collections:
            update_step_status("CT Scan", "running")
            update_status("Generating CT scan data...", 85)
            log_message("Generating CT scan data...", 'info')
            
            if GENERATION_AVAILABLE:
                try:
                    ct_gen = CTScanGenerator()
                    # TODO: Generate CT scan data with config
                    log_message("CT scan generation (placeholder - implement with actual generator)", 'info')
                except Exception as e:
                    log_message(f"Error generating CT scan: {str(e)}", 'error')
            
            update_step_status("CT Scan", "success")
        
        # Populate MongoDB if selected
        if mongo_client and mongo_client.is_connected():
            update_step_status("MongoDB Population", "running")
            update_status("Populating MongoDB...", 90)
            log_message("Populating MongoDB collections...", 'info')
            
            try:
                db = mongo_client.database
                # TODO: Actually insert generated data into MongoDB
                log_message("MongoDB population (placeholder - implement with actual data insertion)", 'info')
                update_step_status("MongoDB Population", "success")
            except Exception as e:
                log_message(f"Error populating MongoDB: {str(e)}", 'error')
                update_step_status("MongoDB Population", "error")
        else:
            log_message("MongoDB not connected - skipping population", 'warning')
        
        log_message("Data generation complete!", 'success')
        update_status("Generation complete!", 100)
        
        # Refresh MongoDB status
        on_refresh_mongo_clicked(None)
        
    except Exception as e:
        log_message(f"Error during generation: {str(e)}", 'error')
        update_status(f"Error: {str(e)}", 0)

def on_clear_preview_clicked(b):
    """Clear all preview visualizations."""
    with stl_hatching_output:
        clear_output(wait=True)
        display(HTML("<p><i>Load an STL file and generate hatching to see visualization</i></p>"))
    
    with laser_params_output:
        clear_output(wait=True)
        display(HTML("<p><i>Generate laser parameters to see distribution plots</i></p>"))
    
    with ispm_output:
        clear_output(wait=True)
        display(HTML("<p><i>Generate ISPM data to see thermal profiles</i></p>"))
    
    with ct_scan_output:
        clear_output(wait=True)
        display(HTML("<p><i>Generate CT scan data to see volume visualization</i></p>"))
    
    with preview_output:
        clear_output(wait=True)
        display(HTML("<p><i>Click 'Preview' to see combined data preview</i></p>"))
    
    log_message("Preview cleared", 'info')

def on_save_config_clicked(b):
    """Save current configuration to JSON file."""
    try:
        config = get_current_config()
        filename = f"generation_config_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        config_path = project_root / 'generation' / 'configs' / filename
        config_path.parent.mkdir(parents=True, exist_ok=True)
        
        with open(config_path, 'w') as f:
            json.dump(config, f, indent=2)
        
        log_message(f"Configuration saved: {filename}", 'success')
        with preview_output:
            clear_output(wait=True)
            display(HTML(f'<p>✅ Configuration saved: <code>{filename}</code></p>'))
    except Exception as e:
        log_message(f"Error saving configuration: {str(e)}", 'error')

def on_refresh_mongo_clicked(b):
    """Refresh MongoDB status display."""
    if not mongo_client:
        mongo_status_html.value = '<b>MongoDB Status</b><br><span style="color:red;">Not connected</span>'
        return
    
    if not mongo_client.is_connected():
        mongo_status_html.value = '<b>MongoDB Status</b><br><span style="color:red;">Not connected</span>'
        return
    
    try:
        # Use the database property (not get_database method)
        db = mongo_client.database
        db_name = mongo_client.config.database
        collections = db.list_collection_names()
        
        status_text = '<b>MongoDB Status</b><br>'
        status_text += f'<b>Database:</b> {db_name}<br>'
        status_text += f'<b>Collections:</b> {len(collections)}<br><ul>'
        
        for coll_name in collections[:10]:  # Show first 10
            count = db[coll_name].count_documents({})
            status_text += f'<li>{coll_name}: {count} documents</li>'
        
        if len(collections) > 10:
            status_text += f'<li>... and {len(collections) - 10} more</li>'
        
        status_text += '</ul>'
        mongo_status_html.value = status_text
        log_message("MongoDB status refreshed", 'success')
    except Exception as e:
        mongo_status_html.value = f'<b>MongoDB Status</b><br><span style="color:red;">Error: {str(e)}</span>'
        log_message(f"Error refreshing MongoDB status: {str(e)}", 'error')

def on_collections_change(change):
    """Update Generate button when collections selection changes."""
    update_generate_button_state()

# ============================================
# Attach All Event Handlers
# ============================================

stl_load_button.on_click(on_stl_load_clicked)
preview_button.on_click(on_preview_clicked)
generate_button.on_click(on_generate_clicked)
clear_preview_button.on_click(on_clear_preview_clicked)
save_config_button.on_click(on_save_config_clicked)
refresh_mongo_button.on_click(on_refresh_mongo_clicked)

# Add observer to STL selector to update button state
def on_stl_selector_change(change):
    """Update Generate button when STL selection changes."""
    update_generate_button_state()

stl_selector.observe(on_stl_selector_change, names='value')

# Add observer to collections selector
collections_to_populate.observe(on_collections_change, names='value')

# Initialize button states
update_generate_button_state()

# Initial MongoDB status refresh
on_refresh_mongo_clicked(None)

print("✅ All event handlers attached and initialized!")
